In [1]:
import datetime

import os

os.getcwd()
if '.git' not in os.listdir():
    os.chdir('./../../../../')
os.listdir()

from ml.ml_core.src.core.model_factory import model_factory

import keras_core as keras
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp


Using TensorFlow backend


2024-07-07 19:12:23.217336: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 19:12:23.217387: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 19:12:23.217407: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-07 19:12:23.223639: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Configurations

In [2]:
params_grid = {
    "model": "lstm",
    "embedder": "camembert",
    "embedding_data": "almanach-camembert-base",
    "dataset": "fr_hf.csv",
    "batch_size": 500
}

In [3]:
data_folder_src = 'ml/ml_core/data/processed/'
ml_core_folder_path = 'ml/ml_core/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'
date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f'deepwoke_{params_grid["embedder"]}_{params_grid["model"]}_{params_grid["dataset"]}_{date_str}'

log_dir = f"{ml_core_folder_path}log/fit/{experiment_name}"
model_weight_dst = f'{ml_core_folder_path}model_weights/{experiment_name}-model.keras'

In [4]:

vector_path = f'{ml_core_folder_path}embedded_vector/{params_grid["embedding_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'

# loading dataset

In [5]:
X = np.load(vector_x_dst)
y = np.load(vector_y_dst)
X = np.expand_dims(X, axis=1)

# Model

## callbacks

In [6]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [7]:

class EarlyStoppingLogging(keras.callbacks.Callback):
    def __init__(self, early_stopping_callback, log_dir):
        super().__init__()
        self.early_stopping = early_stopping_callback
        self.stopped_epoch = 0
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        if self.early_stopping.stopped_epoch > 0:
            self.stopped_epoch = self.early_stopping.stopped_epoch
            with self.writer.as_default():
                tf.summary.scalar('early_stopping_epoch', self.stopped_epoch, step=epoch)
                self.writer.flush()


In [8]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_logging_callback = EarlyStoppingLogging(early_stopping, log_dir)

2024-07-07 19:12:25.316917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 19:12:25.351506: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 19:12:25.351541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 19:12:25.354713: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 19:12:25.354753: I tensorflow/compile

In [9]:
callback = [
    early_stopping,
    tensorboard_callback,
    hp.KerasCallback(log_dir, params_grid),
    early_stopping_logging_callback,
]

# Model building

In [10]:
def rnn_builder(input_dim, output_dim):
    activation_function = 'softmax'
    loss = 'categorical_crossentropy'
    if output_dim == 1:
        activation_function = 'sigmoid'
        loss = 'binary_crossentropy'
    print(input_dim, output_dim)


    input_tensor = keras.layers.Input(shape=input_dim)

    hl = keras.layers.LSTM(64, return_sequences=True, activation='relu')(input_tensor)
    hl = keras.layers.LSTM(128, return_sequences=True, activation='relu')(hl)
    hl = keras.layers.LSTM(256, activation='relu')(hl)
    hl = keras.layers.Dense(256, activation='relu')(hl)
    output_tensor = keras.layers.Dense(output_dim, activation=activation_function)(hl)

    model = keras.models.Model(inputs=[input_tensor], outputs=[output_tensor])
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    return model

### training the model

In [11]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
input_dim = X_train.shape[1:]
model = rnn_builder(input_dim, 1)
print(model.summary)

tf.compat.v1.global_variables_initializer()

(1, 768) 1
<bound method Model.summary of <Functional name=functional_1, built=True>>


In [12]:
model.fit(X_train,
          y_train,
          epochs=500,
          batch_size=params_grid['batch_size'],
          callbacks=callback,
          validation_data=(X_test, y_test)
          )

Epoch 1/500


2024-07-07 19:12:32.322808: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x9954630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-07 19:12:32.322851: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-07 19:12:32.381822: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-07 19:12:32.829242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


 1/33 ━━━━━━━━━━━━━━━━━━━━ 2:25 5s/step - accuracy: 0.5580 - loss: 0.6931

2024-07-07 19:12:33.805795: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


33/33 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - accuracy: 0.7367 - loss: 0.6484 - val_accuracy: 0.7575 - val_loss: 0.5490
Epoch 2/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7595 - loss: 0.5414 - val_accuracy: 0.7575 - val_loss: 0.5323
Epoch 3/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7595 - loss: 0.5215 - val_accuracy: 0.7575 - val_loss: 0.5066
Epoch 4/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7612 - loss: 0.4851 - val_accuracy: 0.7575 - val_loss: 0.4552
Epoch 5/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7707 - loss: 0.4553 - val_accuracy: 0.7977 - val_loss: 0.4469
Epoch 6/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7937 - loss: 0.4474 - val_accuracy: 0.8022 - val_loss: 0.4349
Epoch 7/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7968 - loss: 0.4465 - val_accuracy: 0.8101 - val_loss: 0.4193
Epoch 8/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8075 - loss: 0.4267 - val_accuracy: 0.8112 - val_lo

# save model

In [13]:
model.save(model_weight_dst)